In [41]:
import os
import getpass

os.system('OLLAMA_HOST=0.0.0.0:63321 ollama pull nomic-embed-text')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [42]:
from langchain_community.llms import Ollama 
llm = Ollama(model="mistral", temperature = 0)


In [43]:
llm.invoke("what is LangGraph?")

' LangGraph is a large-scale, multilingual graph database designed for natural language processing (NLP) tasks. It was developed by the Facebook AI Research (FAIR) team and open-sourced in 2019. The primary goal of LangGraph is to provide a scalable solution for storing and querying large amounts of linguistic data, such as words, phrases, and their relationships, across multiple languages.\n\nThe database structure is based on a graph, where nodes represent words or phrases, and edges represent various types of relationships between them, such as synonyms, hypernyms (superclasses), hyponyms (subclasses), antonyms, etc. This allows for efficient querying and analysis of linguistic data, which can be useful in a variety of NLP applications, including machine translation, text summarization, question answering, and more.\n\nLangGraph is built using Apache Spark and Apache Parquet for distributed storage and processing, making it highly scalable and suitable for handling large-scale NLP t

In [ ]:
from langchain_community.document_loaders import WebBaseLoader 

loader = WebBaseLoader(
    web_path=("https://blog.langchain.dev/langgraph/"),
)
docs= loader.load()

In [61]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("D:\RAGwithLangChain\cryptocurrency.pdf")


docs = loader.load()

<>:10: SyntaxWarning: invalid escape sequence '\R'
<>:10: SyntaxWarning: invalid escape sequence '\R'
C:\Users\user\AppData\Local\Temp\ipykernel_14644\737217134.py:10: SyntaxWarning: invalid escape sequence '\R'
  loader = PyPDFLoader("D:\RAGwithLangChain\cryptocurrency.pdf")


In [62]:
docs[0].page_content


'Cooperative Extension Service | Agriculture and Natural Resources | Family and Consumer Sciences | 4-H Youth Development | Community and Economic DevelopmentUniversity of K entucky\nCollege of Agricultur e,\nFood and Environment\nCooper ative Extension Servic eUnderstanding  \nCryptocurrency\nNichole Huff and Kelly May, Family and Consumer SciencesFCS5-475\nThe concept of cryptocurrency , or digital \ncurrency, might sound a bit cryptic to many \nconsumers. As cryptocurrencies become more \nwidely used, consumers should still proceed with \ncaution.\n Cryptocurrency—such as Bitcoin, Litecoin, \nEther, or others—is a form of digital payment con -\nsumers can use to buy goods and services. It exists \nwithout tangible corresponding bills or coins, and \nit is not issued or backed by government agencies. \nAt its core, cryptocurrency is essentially digital \ncomputer software. People or entities can transfer \nit online without the need for a bank or financial \ninstitution. The currency

In [63]:

len(docs[0].page_content)

3215

In [64]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    add_start_index = True,
)

In [65]:
all_splits = text_splitter.split_documents(docs)


In [72]:
len(all_splits)


16

In [73]:
len(all_splits[1].page_content)


980

In [74]:

from langchain_community.embeddings import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model="nomic-embed-text",
)

In [75]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents = all_splits,
    embedding = embedding,
)

In [76]:
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":6}
)

In [78]:
retriever.invoke(
    "What cryptocurrency do you know?"
)

[Document(page_content='Cooperative Extension Service | Agriculture and Natural Resources | Family and Consumer Sciences | 4-H Youth Development | Community and Economic DevelopmentUniversity of K entucky\nCollege of Agricultur e,\nFood and Environment\nCooper ative Extension Servic eUnderstanding  \nCryptocurrency\nNichole Huff and Kelly May, Family and Consumer SciencesFCS5-475\nThe concept of cryptocurrency , or digital \ncurrency, might sound a bit cryptic to many \nconsumers. As cryptocurrencies become more \nwidely used, consumers should still proceed with \ncaution.\n Cryptocurrency—such as Bitcoin, Litecoin, \nEther, or others—is a form of digital payment con -\nsumers can use to buy goods and services. It exists \nwithout tangible corresponding bills or coins, and \nit is not issued or backed by government agencies. \nAt its core, cryptocurrency is essentially digital \ncomputer software. People or entities can transfer \nit online without the need for a bank or financial', me

In [79]:
from langchain import hub 

prompt= hub.pull("rlm/rag-prompt")

In [80]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [81]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt 
    | llm 
    | StrOutputParser()
)

In [83]:
rag_chain.invoke("Where can I buy cryptocurrency?")


" To buy cryptocurrency, you'll first need to create an account on a cryptocurrency trading platform such as Coinbase, Gemini, Binance, Kraken, or eToro. These platforms accept various payment methods like bank transfer, credit card, or money transfer services. It is advisable to check with your bank beforehand to ensure they allow transfers to the platform you choose. Some banks may not permit transactions due to the unregulated nature of cryptocurrencies. Be aware that fees and service offerings vary between platforms, so it's essential to understand the associated costs before making a purchase."